---
# **Criando sistema de alerta para compra e venda de Ações**

---

 **Instalando e importando bibliotecas**

In [ ]:
!pip install plyer

In [ ]:
!pip install MetaTrader

In [ ]:
!pip install mplfinance

In [ ]:
!pip install numpy==1.23.5

In [1]:
from plyer import notification
from time import sleep
from datetime import datetime, timedelta

In [ ]:
import pandas as pd
import MetaTrader5 as mt5
import mplfinance as mpf

**Criando área de notificação com a função [notify](https://plyer.readthedocs.io/en/latest/api.html#plyer.facades.Notification.notify)**

In [ ]:
# Teste do layout da notificação
notification.notify(
    title = 'Teste',
    message = 'A contagem chegou em 5.',
    app_icon = None,
    timeout = 10,
)

In [ ]:
# Incluir um arquivo .ico no alerta (logotipo pra notificação)
icone_imagem = "colocar aqui o caminho até o arquivo .ico"

In [ ]:
# Testando a notificação com uma contagem de 0 a 5 para exibir a mensagem
contagem = 0
for i in range(6):
    if (contagem == 5):
        notification.notify(title = 'Teste',
            message = 'A contagem chegou em 5.',
            app_icon = icone_imagem,
            timeout = 10)
    else:
        contagem+=1
        sleep(1)

**Estrutura e coleta dos dados do alerta**

In [ ]:
# Inicializando o MetaTrader5 e fazendo conexão junto ao Pyhton
mt5.initialize()

Para montar um sistema que busca informações para daytrade, vamos trabalhar com o timeframe de 1 minuto, por isso 5 horas de dados são suficientes. Se fosse um sistema para operações mais longas, precisaríamos de um volume maior de informação.

In [ ]:
data_inicio = datetime.now() - timedelta(hours=5) # Variável para coletar 5 horas de dados a partir de agora
data_fim = datetime.now()
data_fim

In [ ]:
# Coletando informações de preço do mini índice no timeframe 1 min
df_ohlcv = mt5.copy_rates_range('WIN$', mt5.TIMEFRAME_M1, data_inicio, data_fim)
# Criando dataframe
df_ohlcv = pd.DataFrame(df_ohlcv)
# Convertendo coluna 'time' para que seja o índice
df_ohlcv.index = pd.to_datetime(df_ohlcv["time"], unit="s")
# Eliminando colunas desnecessárias
df_ohlcv = df_ohlcv.drop(['time', 'tick_volume', 'spread'], axis=1)
df_ohlcv

**Criação das médias móveis**

In [ ]:
# média móvel exponencial (mme)
df_ohlcv['mm_rapida'] = df_ohlcv.close.ewm(span=9, min_periods=9).mean()
# média móvel simples (mms)
df_ohlcv['mm_lenta'] = df_ohlcv.close.rolling(20).mean()

A média móvel exponencial coloca mais peso nas observações mais recentes, tornando sua reação mais rápida que a média simples.

**Testando visualmente o cruzamento de médias**

*   Usar apenas as últimas 100 linhas a partir do momento presente, para reduzir o processamento.

In [ ]:
# contador que vai pegar as 100 últimas linhas, vai pegar a última e contar pra trás
i = -100
# média rápida = vermelha
fig_mm_rapida = mpf.make_addplot(df_ohlcv['mm_rapida'][i:], type='line', color='red')
# média lenta = azul
fig_mm_lenta = mpf.make_addplot(df_ohlcv['mm_lenta'][i:], type='line', color='blue')
# criando área de plotagem com as 100 últimas linhas do dataframe e as médias móveis
mpf.plot(df_ohlcv[i:], type='candle', addplot=[fig_mm_rapida, fig_mm_lenta], figsize=(10,6))

**Função para detecção de cruzamento das médias móveis**
*   Agora vamos unir todos os passos acima em uma função de detecção de sinal de trade.          


In [ ]:
def cruzamento_mms(data_series, window_mm1=9, window_mm2=20, tipo_mm1='mme', tipo_mm2='mma'):

    """
    Verifica o cruzamento de duas médias móveis.

    :param data_series (pd.core.series.Series): série de dados (usualmente fechamento dos ativos);
    :param window_mm1 (int): # períodos da primeira média móvel (média mais rápida). Default 9;
    :param window_mm2 (int): # períodos da segunda média móvel (média mais lenta). Default 20;
    :param tipo_mm1 (str): tipo da média móvel rápida. Possibilidades: média móvel aritmética (mma) ou média móvel exponencial (mme). Default 'mme';
    :param tipo_mm2 (str): tipo da média móvel lenta. Possibilidades: média móvel aritmética (mma) ou média móvel exponencial (mme). Default 'mma';

    :return (bool): True or False quando houver cruzamentos
    """

    if tipo_mm1=='mme':
        mm_rapida = data_series.ewm(span=window_mm1, min_periods=window_mm1).mean()
    else:
        mm_rapida = data_series.close.rolling(window_mm1).mean()

    if tipo_mm2=='mme':
        mm_lenta = data_series.ewm(span=window_mm2, min_periods=window_mm2).mean()
    else:
        mm_lenta = data_series.rolling(window_mm2).mean()

    cond_cruz_acima = (mm_rapida[-1] > mm_lenta[-1]) and (mm_rapida[-2] < mm_lenta[-2])
    cond_cruz_abaixo = (mm_rapida[-1] < mm_lenta[-1]) and (mm_rapida[-2] > mm_lenta[-2])

    if cond_cruz_acima:
        return cond_cruz_acima, 'sinal de compra'
    elif cond_cruz_abaixo:
        return cond_cruz_abaixo, 'sinal de venda'
    else:
        return False, 'nada a fazer'

In [ ]:
cruzamento_mms(df_ohlcv.close, window_mm1=9, window_mm2=20, tipo_mm1='mme', tipo_mm2='mma')

**Alerta com os dados real time do MT5**
*   Até o momento, testamos somente com dados do Mini Índice, agora testaremos com todas as Ações do IBOV em tempo real.

In [ ]:
tickers_ibov = ['ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3',
                'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3',
                'CIEL3', 'CMIG4', 'CMIN3', 'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3',
                'CVCB3', 'CYRE3', 'DXCO3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENEV3', 'ENGI11',
                'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3', 'HYPE3', 'IGTI11',
                'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3',
                'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PETZ3', 'PRIO3', 'RADL3',
                'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3', 'RRRP3', 'SANB11', 'SBSP3', 'SLCE3', 'SMTO3',
                'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5', 'VALE3', 'VBBR3',
                'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

Faremos um loop de repetição, que permanecerá ativo enquanto as condições estiverem sendo satisfeitas (sejam verdadeiras).

No caso, enquanto o horário for menor que 16 horas, para cada Ação(ticker):
*   Capturar dados de preço das Ações das últimas 5 horas, no timeframe 5 min;
*   Criar um dataframe;
*   Converter a coluna 'time' para que seja o índice;
*   Executar a função de verificação de cruzamento de médias;
*   Emitir aviso de sinal de compra e venda caso encontre cruzamento de médias.

In [ ]:
# defininda valores para médias móveis
window_mm1 = 9
window_mm2 = 20

# início do loop de repetição
while datetime.now().hour < 16:
    # verificação das Ações
    for i in tickers_ibov:
        # coleta e tratamento de dados da Ação
        data_inicio = datetime.now() - timedelta(hours=5)
        data_fim = datetime.now()
        df_ohlcv = mt5.copy_rates_range(i, mt5.TIMEFRAME_M5, data_inicio, data_fim)
        df_ohlcv = pd.DataFrame(df_ohlcv)
        df_ohlcv.index = pd.to_datetime(df_ohlcv["time"], unit="s")

        # excecução da função de cruzamento de médias
        sinal, aviso = cruzamento_mms(df_ohlcv.close, window_mm1, window_mm2)

        if sinal and aviso=='sinal de compra':
            # envio da notificação de compra para o PC
            notification.notify(title = 'Alert de Compra',
                message = f'Cruzamento para cima em {i} !',
                app_icon = icone_imagem,
                timeout = 2)
            print(i, 'Sinal de Compra')

        if sinal and aviso=='sinal de venda':
            # envio da notificação de venda para o PC
            notification.notify(title = 'Alerta de Venda',
                message = f'Cruzamento para baixo em {i} !',
                app_icon = icone_imagem,
                timeout = 2)
            print(i, 'Sinal de Venda')

        # finalização da verificação das Ações, vai pausar por 30s e retomar até que seja 16h
        if i=='YDUQ3':
            print('Varredura finalizada, reiniciando em 30s')
            sleep(30)

Importante salienter que se não faço operações em todos os ativos do IBOV, não há motivos para que estes estejam na lista de análise, isso vai onerar o processamento da máquina.

Para minminar o processamento, posso executar mais algum screening prévio à análise, de forma a filtrar mais os ativos que serão analisados.